# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [61]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/data2.sqlite")

In [62]:
# Your code here; create the connection

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [63]:
# Your code here
q0 = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
)
;
"""
q0_result = pd.read_sql(q0, connec)
q0

"\nSELECT\n    customerNumber,\n    contactLastName,\n    contactFirstName\nFROM customers\nWHERE customerNumber IN (\n    SELECT customerNumber\n    FROM orders\n    WHERE orderDate = '2003-01-31'\n)\n;\n"

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [67]:
# Your code here
q1 = '''
SELECT p.productCode, p.productName, SUM(od.quantityOrdered) as totalQuantity
FROM products p 
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName
ORDER BY totalQuantity DESC
;
'''

q1_result = pd.read_sql(q1, conn)
q1_result

,productCode,productName,totalQuantity
0,S18_3232,1992 Ferrari 360 Spider red,1808
1,S18_1342,1937 Lincoln Berline,1111
2,S700_4002,American Airlines: MD-11S,1085
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076
4,S50_1341,1930 Buick Marquette Phaeton,1074
...,...,...,...
104,S18_2870,1999 Indy 500 Monte Carlo SS,855
105,S18_2248,1911 Ford Town Car,832
106,S24_3969,1936 Mercedes Benz 500k Roadster,824
107,S24_1046,1970 Chevy Chevelle SS 454,803


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [88]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q2 = """
SELECT DISTINCT(quantityOrdered) DESC
FROM products
;
"""
query = "SELECT DISTINCT * FROM orderDetails"
q2_results = pd.read_sql_query(query, conn)

q2_results

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4
...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9
2992,10425,S24_2840,31,31.82,5
2993,10425,S32_1268,41,83.79,11
2994,10425,S32_2509,11,50.32,6


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [100]:
# Your code here
q3 = """
SELECT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o2 ON c.customerNumber = o2.customerNumber
JOIN orderdetails od ON o2.orderNumber = od.orderNumber
WHERE od.productCode IN (
SELECT productCode
FROM orderdetails
GROUP BY productCode
HAVING COUNT(DISTINCT quantityOrdered) < 20);
"""

q3_results = pd.read_sql(q3, connec)
q3_results

,employeeNumber,firstName,lastName,city,officeCode
0,1216,Steve,Patterson,Boston,2
1,1216,Steve,Patterson,Boston,2
2,1216,Steve,Patterson,Boston,2
3,1504,Barry,Jones,London,7
4,1504,Barry,Jones,London,7
...,...,...,...,...,...
2141,1370,Gerard,Hernandez,Paris,4
2142,1370,Gerard,Hernandez,Paris,4
2143,1370,Gerard,Hernandez,Paris,4
2144,1370,Gerard,Hernandez,Paris,4


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [101]:
# Your code here
q4 = """
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber)
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING AVG(c.creditLimit) > 15000.00
;
"""

q4_results = pd.read_sql(q4, connec)
q4_results

,employeeNumber,firstName,lastName,COUNT(c.customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!